# **Spotify Web API Task 1 : Obtaining an OAuth Token**

### **Background** ###
This notebook provides a step-by-step guide to obtaining an OAuth token from the Spotify Web API. An OAuth token is necessary to authenticate requests and access specific data on Spotify, such as creating and modifying playlists on your account.

### **Prerequisites**
Before proceeding, you need to set up a Spotify Developer App and obtain the following details:

- **Client ID**: Your application's unique identifier.

- **Client Secret**: A secret key for your application, used to authenticate your requests.

- **Redirect URI**: A URL to which Spotify will redirect after user authorization. This must be registered in your Spotify Developer Dashboard.
<br>


These credentials can be found on your app's dashboard on the Spotify Developer Portal. Make sure to never hardcode your client ID and client secret directly in your scripts due to security risks. Instead, store them securely, such as in an environment file (.env), and load them into your program as needed.
<br>

### **User Authentication**
To obtain an OAuth token, you will need to log in to your Spotify account and grant permission for the app to access your data. This is required to authenticate and authorize your requests. The program will ask for the following... 

- **Username or email address :** The users Spotify username or email address 
- **User ID :** The users Spotify User ID, this can be found by checking your profile URI on the Spotify Web Application. 
- **Password :** The users Spotify password 


### **Scope of Access**
The OAuth token you obtain through this process will allow your application to create and modify playlists on your Spotify account. Depending on the permissions (scopes) you request, you may have access to additional features. Make sure to review and request only the scopes your application needs.For this request our scope is 'playlist-modify-private playlist-modify-public' as we want to use this token in the next task, where we will be creating a playlist and appending songs to it.

### **Step by Step Guide**

Here is a step by step guide to the process :

1) **Import Libraries :** Importing the relevant libraries.

2) **Import Secrets :** Initialise the .env file and retrieve sensitive information from within.

3) **Make HTTP Headers :** Set up the Spotify Authorisation Header Credentials 

4) **Selenium to Automate Log In :** Set up browser environment, open Spotify Authorisation page, and ask user to input log in details. 

6) **Parse Response for Authorisation Code :** Obtain and parse through response URL to obtain authorisation code.

7) **Exchange Authorisation Code :** Access the Spotify Web API, make a POST request and obtain JSON response. 

8) **Obtain OAuth and Refresh Tokens :** Parsing through JSON response to obtain tokens. 

9) **Export Tokens to ENV File :** Export the tokens to .env file 

By the end of this notebook, you will have a clear understanding of how to obtain an OAuth token to the Spotify Web API

In [106]:
# Imports 
from dotenv import load_dotenv
import os 
import base64
import urllib.parse
import getpass
from selenium import webdriver
from selenium.webdriver.common.by import By
from http.server import BaseHTTPRequestHandler
import requests

In [94]:
# Initialise the secrets within .env file 
load_dotenv()
client_id = os.getenv('CLIENT_ID')
client_secret = os.getenv('CLIENT_SECRET')

In [95]:
# Spotify API credentials 
client_secret # This is specific to the app you are building and can be found in the Spotify API Developer Dashboard
client_id # This is specific to the app you are building and can be found in the Spotify API Developer Dashboard
redirect_uri = 'http://localhost:8888/callback'  # Must match the redirect URI set in the apps Spotify Developer Dashboard
scope = 'playlist-modify-private playlist-modify-public'  # These are the scopes needed for our app as we need to create playlist 
                                                          # Change based on your project needs

# Direct to Spotify's authorization URL
auth_url = (
    "https://accounts.spotify.com/authorize"
    "?response_type=code"
    f"&client_id={client_id}"
    f"&redirect_uri={urllib.parse.quote(redirect_uri)}"
    f"&scope={urllib.parse.quote(scope)}"
)

# Opens a chrome window for the user to sign in with Spotify Log in and Password
driver = webdriver.Chrome()
driver.get(auth_url)

In [96]:
# Locate the username input field using the 'id' attribute
input_field = driver.find_element(By.ID, "login-username")
# Click into the username field 
input_field.click()
# Ask the user to input their Spotify username 
input_field.send_keys(input("Please enter your Spotify Username or Password"))
spotify_user_id = input('What is your Spotify USER ID?')

In [97]:
# Locate the password input field using the 'id' attribute
password_field = driver.find_element(By.ID, "login-password")
# Click into the password field
password_field.click()
# Ask the user to input their Spotify password 
password_field.send_keys(getpass.getpass(prompt='Please enter your password'))

In [98]:
# Locate the login button using the 'id' attribute
login_button = driver.find_element(By.ID, "login-button")
# Click the login button
login_button.click()

In [99]:
# Get the URL of the page once user is logged in 
current_url = driver.current_url
print(current_url)

# Parse the url in order to get authorisation code 
parsed_url = urllib.parse.urlparse(current_url)
auth_code = urllib.parse.parse_qs(parsed_url.query).get('code')
if not auth_code:
    print("Error: Authorization code not found in the redirect URL.")
else:
    auth_code = auth_code[0]

http://localhost:8888/callback?code=AQA3RqLqVNGR_KykoAgzoPIwOsXBxZJACVRZbkGPkTQ6AVDzTSfU7C4Y6aJhgYF3v0ReYRCtI33nk0IRijNQ3KOH2z0D9IrhcIusg_lmN4u1a4HpHDcHtyc-Q6XEW-YgX5-_w7hRTJTnBPw-95JsYEhtI9v5q77jUjsXR2q1cG2CY48zvhDExRWepMwZOnW1ae7phUDzJfAuBMSHXBs6psscsB5i2rcvwq2XZ99VhGjjvd0


In [100]:
# Exchange the authorization code for an OAuth token

# URL of the Spotify API endpoint which issues OAuth tokens
token_url = 'https://accounts.spotify.com/api/token'
# Encode client credentials (Base64 encoding), Spotify's token endpoint expects this in the 'Authorization' header
client_credentials = f"{client_id}:{client_secret}"
encoded_credentials = base64.b64encode(client_credentials.encode()).decode()

# HTTP Headers are neccessary to authenicate and specify the request 
# Content type specifies the format of the data being sent to the API
headers = {
'Authorization': f'Basic {encoded_credentials}',
 'Content-Type': 'application/x-www-form-urlencoded'
}
# Specifies the method we are using to obtain the OAuth token  
data = {
'grant_type': 'authorization_code',
'code': auth_code,
'redirect_uri': redirect_uri
    }

# Make the POST request to get the new access token
response = requests.post(token_url, headers=headers, data=data)

In [101]:
# Checking the request was successful and obtaining the access and refresh tokens 
if response.status_code == 200:
        token_info = response.json()
        access_token = token_info['access_token']
        refresh_token = token_info['refresh_token']
        print("Access Token:", access_token)
        print("Refresh Token:", refresh_token)
else:
        print("Failed to get access token:", response.status_code, response.text)

Access Token: BQCSunvs0LH6bd2yYXemQNaYWjEMb97w-2ncqImJoD6niQ0rHzy9O19nWmJOi2aND9QNx2T-P7ta_D4G1_UVegOip1WZz5hq0RuF6myvN_iU51UgsOXvc2WIDYerr2TtaoyL59LyahWWigy1kNhQ6cejItfLop7CXq9rwgMoAhoLdHQ0JkUpczP5y1bfGPB6VqVTIMB6uEZ--Vjv7ovywEeYc7M1Psy_qjYGmdsLIHLhfTgsiatCIcs
Refresh Token: AQBo1wBpdi89pBTsawe1LUY5iISrJU7wdQvARr9DMlRMSL7mv9Z6yy2oZ_f_oPuCAh3dMVjqSRDiHgsJV7Z4lMJnbnGJQJFpD_hpLIUbWG7Utc05j1xa2ARo_f0etqKLqGU


In [104]:
# Path to your .env file
env_file_path = ".env"

In [105]:
# Write the token to the .env file
with open(env_file_path, "w") as env_file:
    env_file.write(f"OAUTH_TOKEN ='{access_token}'\nREFRESH_TOKEN ='{refresh_token}'\nCLIENT_ID ='{client_id}'\nCLIENT_SECRET ='{client_secret}'\nUSER_ID ='{spotify_user_id}'")